In [1]:
import pandas as pd
import requests,json,os
from urllib.parse import quote 
from datetime import datetime

In [2]:
with open('key/sk_open_api_key.txt') as f_:
    sk_key = f_.read()

In [26]:
len(sk_key) 

40

In [32]:
target = '서울역'
timep = '08:40'
hh, mm = timep.split(":")
df_read = pd.read_csv('stn_code.csv')
stn = '수원역'  # 입력할 역
stn_code = df_read[df_read['stationName'] == target]['stationCode'].values[0]
stn_code

'133'

In [33]:
# 혼잡도
station_code = stn_code
dow = 'MON' # 대문자
# hh = '08'
url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/stat/train/stations/{station_code}?dow={dow}&hh={hh}"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": sk_key
}

response = requests.get(url, headers=headers).json()


In [29]:
response.keys()

dict_keys(['status', 'contents'])

In [74]:
response['contents'].keys()

dict_keys(['subwayLine', 'stationName', 'stationCode', 'stat', 'statStartDate', 'statEndDate'])

In [75]:
len(response['contents']['stat'])

34

In [76]:
response['contents']['stat'][0]['startStationName']

'의정부역'

In [77]:
response['contents']['stationName']

'수원역'

In [89]:
response['contents']['stat'][15]['data']

[{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '10', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '30', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '50', 'congestionTrain': 0}]

In [79]:
temp_list = []
temp_list2 = []
for stat in response['contents']['stat']:
    if stat['updnLine'] == 1:
        temp_list.append(pd.DataFrame(stat['data']))
    elif stat['updnLine'] == 0:
        temp_list2.append(pd.DataFrame(stat['data']))
# temp_list = temp_list[0]
# temp_list
# temp_list2 = temp_list2[0]
# temp_list2


In [34]:
data1 = [pd.DataFrame(stat['data']) for stat in response['contents']['stat'] if stat['updnLine'] == 1]
data2 = [pd.DataFrame(stat['data']) for stat in response['contents']['stat'] if stat['updnLine'] == 0]

In [80]:
df = pd.concat(temp_list)
df['updn'] = 1
df_ = pd.concat(temp_list2)
df_['updn'] = 0
df = pd.concat((df,df_))
df

,dow,hh,mm,congestionTrain,updn
0,MON,08,00,0,1
1,MON,08,10,0,1
2,MON,08,20,0,1
3,MON,08,30,8,1
4,MON,08,40,8,1
...,...,...,...,...,...
1,MON,08,10,0,0
2,MON,08,20,0,0
3,MON,08,30,0,0
4,MON,08,40,0,0


In [115]:
# 최대혼잡도
df_res = df[df['congestionTrain'] != 0].copy().reset_index(drop=True)
df_res.drop(columns=['dow','hh','mm'],inplace=True)


In [116]:
df_res = df_res.groupby('updn')['congestionTrain'].agg('max').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,15
1,1,32


In [109]:
df_res = df_res.groupby('updn')['congestionTrain'].agg('min').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,4
1,1,6


In [35]:
df = pd.concat(data1)
df['updn'] = 1
df_ = pd.concat(data2)
df_['updn'] = 0
df = pd.concat((df,df_))
df

,dow,hh,mm,congestionTrain,updn
0,MON,08,00,0,1
1,MON,08,10,22,1
2,MON,08,20,44,1
3,MON,08,30,44,1
4,MON,08,40,22,1
...,...,...,...,...,...
1,MON,08,10,183,0
2,MON,08,20,183,0
3,MON,08,30,191,0
4,MON,08,40,191,0


In [29]:
mm = 10

In [36]:
df_res = df[df['congestionTrain'] != 0].copy()
df_res = df_res[df_res['mm'] == mm].copy()
df_res.drop(columns=['dow','hh','mm'],inplace=True)
df_res = df.groupby('updn')['congestionTrain'].agg('max').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,197
1,1,69


In [123]:
df_res.values

array([[ 0, 15],
       [ 1, 32]], dtype=int64)

In [31]:
#######################
import requests

url = "https://apis.openapi.sk.com/puzzle/subway/meta/stations?type=train"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": sk_key
}

response = requests.get(url, headers=headers).json()

In [32]:
response.keys()

dict_keys(['status', 'contents'])

In [34]:
re = response['contents']

In [35]:
df = pd.DataFrame(re)

In [36]:
df

,subwayLine,stationName,stationCode
0,1호선,소요산역,100
1,1호선,동두천역,101
2,1호선,보산역,102
3,1호선,동두천중앙역,103
4,1호선,지행역,104
...,...,...,...
477,5호선,방이역,P551
478,5호선,오금역,P552
479,5호선,개롱역,P553
480,5호선,거여역,P554


In [37]:
df.to_csv('stn_code.csv',index=False)

In [50]:
stn = '소요산역'
selected_code = df[df['stationName'] == stn]['stationCode'].values[0]
selected_code

'100'

In [41]:
df.loc[df['stationName'] == stn, 'stationCode']

Series([], Name: stationCode, dtype: object)

In [ ]:
def get_stn_code():
    url = "https://apis.openapi.sk.com/puzzle/subway/meta/stations?type=train"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "appkey": sk_key
    }
    response = requests.get(url, headers=headers).json()
    re = response['contents']
    df = pd.DataFrame(re)
    df.to_csv('stn_code.csv',index=False)
    return None
    

In [38]:
weekdays = ["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"]
today = datetime.today().weekday()
dow = weekdays[today]

In [3]:
# def create_stn_code():
#     url = "https://apis.openapi.sk.com/puzzle/subway/meta/stations?type=train"
#     headers = {
#         "accept": "application/json",
#         "Content-Type": "application/json",
#         "appkey": sk_key
#     }
#     response = requests.get(url, headers=headers).json()
#     re = response['contents']
#     df = pd.DataFrame(re)
#     df.to_csv(f'stn_code.csv',index=False)
#     return None
def get_stn_code(station):
    df_read = pd.read_csv(f'stn_code.csv')
    stn = station  # 입력할 역
    stn_code = df_read[df_read['stationName'] == stn]['stationCode'].values[0]
    return stn_code
# 역명, 요일, 시간, 분(10분단위)를 입력하면 입력한 시간에 해당되는 상하행별 최대 혼잡도 반환
def get_cong(station,hh,mm):
    sk_key_ = 'key/sk_open_api_key.txt'
    # 고유코드들을 구한 csv파일이 없으면 고유코드들을 구하는 함수 실행
    # if not os.path.exists(stn_code_file_path):
    #     create_stn_code()
    weekdays = ["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"]
    today = datetime.today().weekday()
    dow = weekdays[today]
    
    station_code = get_stn_code(station)
    url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/stat/train/stations/{station_code}?dow={dow}&hh={hh}"
    # url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/stat/train/stations/{station_code}?dow=MON&hh={hh}"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "appkey": sk_key
    }

    response = requests.get(url, headers=headers).json()
    # data1에는 상행선들의 데이터
    # data2에는 하행선들의 데이터
    data1 = [pd.DataFrame(stat['data']) for stat in response['contents']['stat'] if stat['updnLine'] == 1]
    data2 = [pd.DataFrame(stat['data']) for stat in response['contents']['stat'] if stat['updnLine'] == 0]
    # 리스트에 저장된 데이터프레임들을 하나로 만듬
    df = pd.concat(data1)
    df['updn'] = 1
    df_ = pd.concat(data2)
    df_['updn'] = 0
    # 합쳐진 상하행선 데이터들을 합침
    df = pd.concat((df,df_))
    # 특수한 사유로 혼잡도 0을 갖는 데이터들이 있는데 이 데이터들은 포함하지 않음
    df_res = df[df['congestionTrain'] != 0].copy()
    # 그 시간대에서 원하는 분(10분단위)를 고름
    df_res = df_res[df_res['mm'] == int(mm)].copy()
    # 필요없는 부분을 제거하고 그룹바이
    df_res.drop(columns=['dow','hh','mm'],inplace=True)
    df_res = df.groupby('updn')['congestionTrain'].agg('max').reset_index().round(2)
    return df_res.values

In [12]:
target = '서울역'
timep = '08:40'
hh, mm = timep.split(":")
a = get_cong(station=target,hh=hh,mm=mm)
a

array([[  0, 180],
       [  1,  65]], dtype=int64)

In [9]:
dn = int(a[0][1])
up = int(a[1][1])
dn, up

(180, 65)

In [11]:
dn_img = 1 if dn < 20 else 2 if dn < 40 and dn>=20 else 3 if dn < 80 and dn>=40 else 4 if dn < 140 and dn>=80 else 5
up_img = 1 if up < 20 else 2 if up < 40 and up>=20 else 3 if up < 80 and up>=40 else 4 if up < 140 and up>=80 else 5
dn_img,up_img

(5, 3)

In [ ]:
def show_cong(timep, target, app):
    hh, mm = timep.split(":")
    cong = get_cong(station=target, hh=hh, mm=mm)
    dn = int(cong[0][1])
    up = int(cong[1][1])
    dn_img = 1 if dn < 20 else 2 if dn < 40 else 3 if dn < 80 else 4 if dn < 140 else 5
    up_img = 1 if up < 20 else 2 if up < 40 else 3 if up < 80 else 4 if up < 140 else 5
